# Chapter 5 - Model Development of An Close-loop PI Controller

## Introduction

In previous chapter, you have gone through the model development of an open-loop PI controller.

In this chapter, you will develop a clsoe-loop PI controller ``CLPI`` that feedbacks to the system by extending ``OLPI``.

## Objective

In this exercise, you will:
- Develop an close-loop PI controller

## Submission

After complete the chapter, please save and submit this jupyter notebook file in **CANVAS** with **FORMATTED** name:

`FirstName_LastName_NetID_ChID.ipynb`, for example, `Tim_Cook_tcook3_Ch5.ipynb`.

## Model Development

In this chapter, you will extend the existing model ``OLPI`` as an close-loop PI controller ``CLPI``.

### Model Design

The model ``CLPI`` is an clsoe-loop PI controller that takes Generator speed deviation `wd` as input, and feedbacks to Turbine Governor power reference ``pref``.

In such way, the model ``CLPI`` aims to regualte the Generator speed deviation ``wd``. Therefore, the model ``CLPI`` is actually a secondary frequency regulator.

### Working Branch

You can continue to work on the branch ``olpi``.

### Model Implementation

The model code is located in the directory ``$HOME/andes/andes/models/misc/clpi.py``.

#### CLPIData

Model ``CLPI`` holds the same data as model ``OLPI``.

#### CLPIModel

Compared to ``OLPIModel``, ``CLPIModel`` needs an ``ExtAlgeb`` to feedback the PI controller output to the Turbine Governor power reference. 

```python
class CLPIModel(Model):
    """
    Implementation for close-loop PI controller.
    """

    def __init__(self, system, config):
        Model.__init__(self, system, config)
        self.group = 'Experimental'
        self.flags.tds = True

        self.wd = ExtAlgeb(model='TurbineGov', src='pout', indexer=self.gov,
                           info='Generator speed deviation',
                           unit='p.u.',
                           tex_name=r'\omega_{dev}',
                           )
        self.pout = ExtAlgeb(model='TurbineGov', src='pout', indexer=self.gov,
                             tex_name='P_{out}',
                             info='Turbine governor output',
                             )
        self.pout0 = ConstService(v_str='pout',
                                  tex_name='P_{out0}',
                                  info='initial turbine governor output',
                                  )
        self.PID = PIDController(u=self.wd, kp=self.kP, ki=self.kI,
                                 kd=self.kD, Td=self.tD,
                                 tex_name='PID', info='PID', name='PID',
                                 ref=self.pout0,
                                 )
        self.pref = ExtAlgeb(indexer=self.gov,
                             tex_name='P_{ref}',
                             info='Turbine governor output',
                             # model=<**ANSWER**>,
                             # src=<**ANSWER**>,
                             # e_str=<**ANSWER**>,
                             # v_str=<**ANSWER**>,
                             )
```
In ``CLPIModel``, ``pref`` is the ***feedback*** to the Turbine Governor.

#### Model Finalization

Similar to assembling ``OLPI``, you can assemble ``CLPIData`` and ``CLPIModel`` into ``CLPI``.

### Model Test

Since ``CLPI`` takes exactly the same parameters input as ``OLPI``, you can revise the test case ``ieee14_olpi.xlsx`` by changing the model name from ``OLPI`` to ``CLPI``.

In [ ]:
import matplotlib.pyplot as plt
import andes
andes.config_logger(stream_level=20)

%matplotlib inline


In [ ]:
ss = andes.run(andes.get_case('ieee14/ieee14_clpi.xlsx'),
               default_config=True,
               no_output=True,
               setup=False)


In [ ]:
ss.add('Toggle', dict(model="GENROU", dev='GENROU_5', t=1.0))

In [ ]:
ss.setup()

In [ ]:
ss.Toggle.u.v[[0, 1]] = 0

In [ ]:
ss.CLPI.as_df()

In [ ]:
ss.PFlow.run()

In [ ]:
ss_init = ss.TDS.init()

In [ ]:
ss.TDS.config.tf = 20
ss.TDS.config.criteria = 0
ss.TDS.run()

Here we rerun the open-loop PI controller test case as a comparasion.

In [ ]:
ss_ol = andes.run(andes.get_case('ieee14/ieee14_olpi.xlsx'),
                  default_config=True,
                  no_output=True,
                  setup=False)

ss_ol.add('Toggle', dict(model="GENROU", dev='GENROU_5', t=1.0))
ss_ol.setup()
ss_ol.Toggle.u.v[[0, 1]] = 0

ss_ol.PFlow.run()
ss_ol_init = ss_ol.TDS.init()

ss_ol.TDS.config.tf = 20
ss_ol.TDS.config.criteria = 0
ss_ol.TDS.run()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(10, 8))
plt.subplots_adjust(wspace=0.3, hspace=0.3)

cl = 'Close-loop PID'
ol = 'Open-loop PID'

ss.TDS.plt.plot(ss.TGOV1.pout,
                a=(0), yheader=[cl],
                color=['black'], linestyles=['-'],
                fig=fig, ax=ax[0, 0], show=False,
                grid=True,
                )
ss.TDS.plt.plot(ss.GENROU.omega,
                a=(0), yheader=[cl],
                color=['black'], linestyles=['-'],
                fig=fig, ax=ax[0, 1], show=False,
                grid=True, ytimes=60, ymin=59.6,
                )
ss.TDS.plt.plot(ss.CLPI.PID.u,
                yheader=[cl],
                color=['black'], linestyles=['-'],
                fig=fig, ax=ax[1, 0], show=False,
                grid=True,
                )
ss.TDS.plt.plot(ss.CLPI.PID_y,
                yheader=[cl],
                color=['black'], linestyles=['-'],
                fig=fig, ax=ax[1, 1], show=False,
                grid=True,
                )

ss_ol.TDS.plt.plot(ss_ol.TGOV1.pout,
                   a=(0), yheader=[ol],
                   color=['red'], linestyles=['--'],
                   fig=fig, ax=ax[0, 0], show=False,
                   grid=True,
                   title='TGOV1_1 Output Power',
                   ylabel='Active Power [p.u.]',
                   )
ss_ol.TDS.plt.plot(ss_ol.GENROU.omega,
                   a=(0), yheader=[ol],
                   color=['red'], linestyles=['--'],
                   fig=fig, ax=ax[0, 1], show=False,
                   grid=True, ytimes=60,
                   title='GENROU_1 Frequency',
                   ylabel='Frequency [Hz]',
                   )
ss_ol.TDS.plt.plot(ss_ol.OLPI.PID.u,
                   yheader=[ol, 'Ref'],
                   color=['red'], linestyles=['-'],
                   fig=fig, ax=ax[1, 0], show=False,
                   grid=True,
                   title='PID Input',
                   )
ss_ol.TDS.plt.plot(ss_ol.OLPI.PID_y,
                   yheader=[ol],
                   color=['red'], linestyles=['--'],
                   fig=fig, ax=ax[1, 1], show=False,
                   grid=True,
                   title='PID Output',
                   )

From the above figure "TGOV1_1 Output Power", it can be seen that compared to ``OLPI``, ``CLPI`` is able to regulate the Generator speed deviation ``wd``.

As shown in "GENROU1_1 Frequency", although the ``CLPI`` is actually a neither ``OLPI`` nor ``CLPI`` is able to regulate the frequency to nominal value ``60``Hz. This is because the TGOV1_1 has reached the upper limiter ``VMAX``, as shown below, the first value of flag ``TGOV1.LAG_lim.zu`` is ``1``.

In [ ]:
ss.TGOV1.LAG_lim.zu